## Load libraries

In [ ]:
using HTTP
using JSON
using DataStructures

## Set token/URL

In [ ]:
token = get(ENV, "API_TOKEN", "")
@assert !isempty(token)
url = get(ENV, "API_BASE_URL", "https://covid-modelling-stg.epcc.ed.ac.uk")

In [ ]:
headers = OrderedDict("accept"=>"application/json; charset=utf-8", "Content-Type"=>"application/json", "Authorization"=> "Bearer "*token);

## Input

In [ ]:
inputJSON = OrderedDict(
  "model_slug" => "sir-ode-python",
  "config" => Dict(
    "p" => [
      0.5,
      0.25
    ],
    "u0" => [
      0.99,
      0.01,
      0
    ],
    "tspan" => [
      0,
      10000
    ]
  )
);

In [ ]:
stp = 60
timeout = 600;

## Post simulations

In [ ]:
post_simulations_response = HTTP.post(url * "/api/simulations", headers, json(inputJSON))

In [ ]:
HTTP.status(post_simulations_response)

In [ ]:
post_simulations_content = JSON.parse(String(HTTP.body(post_simulations_response)))

## Check simulations

In [ ]:
check_simulations_url = url * "/api/simulations/" * string(post_simulations_content["id"])

In [ ]:
check_simulations_response = HTTP.get(check_simulations_url, headers)

In [ ]:
HTTP.status(check_simulations_response)

In [ ]:
check_simulations_content = JSON.parse(String(HTTP.body(check_simulations_response)))

In [ ]:
check_simulations_content["model_runs"]

## Export simulation output

In [ ]:
export_simulations_params = OrderedDict(
    "model" => inputJSON["model_slug"],
    "format" => "results"
)

In [ ]:
export_simulations_url =  url * "/api/simulations/" *  string(post_simulations_content["id"]) * "/export"
export_simulations_url

In [ ]:
tm = 0
export_simulations_response = nothing
while tm <= timeout
    export_simulations_response = HTTP.get(export_simulations_url, headers, query = export_simulations_params, status_exception = false)
    if HTTP.status(export_simulations_response)==200
        break
    end
    sleep(stp)
    tm += stp
end

In [ ]:
check_simulations_content = JSON.parse(String(HTTP.body(export_simulations_response)))